In [1]:
import tensorflow as tf

In [2]:
a = tf.random_uniform([1])
b = tf.random_normal([1])

print("Session 1")
with tf.Session() as sess1:
  print(sess1.run(a))  # generates 'A1'
  print(sess1.run(a))  # generates 'A2'
  print(sess1.run(b))  # generates 'B1'
  print(sess1.run(b))  # generates 'B2'

print("Session 2")
with tf.Session() as sess2:
  print(sess2.run(a))  # generates 'A3'
  print(sess2.run(a))  # generates 'A4'
  print(sess2.run(b))  # generates 'B3'
  print(sess2.run(b))  # generates 'B4'


Session 1
[0.08207929]
[0.93055236]
[0.39524803]
[-0.32118896]
Session 2
[0.19853282]
[0.85868704]
[0.5585983]
[-1.0956829]


In [3]:
tf.set_random_seed(12)
a = tf.random_uniform([1]) 
b = tf.random_normal([1])
# Repeatedly running this block with the same graph will generate the same
# sequence of values for 'a', but different sequences of values for 'b'.
print("Session 1")
with tf.Session() as sess1:
  print(sess1.run(a))  # generates 'A1'
  print(sess1.run(a))  # generates 'A2'
  print(sess1.run(b))  # generates 'B1'
  print(sess1.run(b))  # generates 'B2'

print("Session 2")
with tf.Session() as sess2:
  print(sess2.run(a))  # generates 'A1'
  print(sess2.run(a))  # generates 'A2'
  print(sess2.run(b))  # generates 'B3'
  print(sess2.run(b))  # generates 'B4'

Session 1
[0.5112722]
[0.35797966]
[-0.52159685]
[0.581533]
Session 2
[0.5112722]
[0.35797966]
[-0.52159685]
[0.581533]


In [1]:
#coding:utf-8 
  
# 学习使用 tf.app.flags 使用，全局变量 
# 可以再命令行中运行也是比较方便，如果只写 python app_flags.py 则代码运行时默认程序里面设置的默认设置 
# 若 python app_flags.py --train_data_path <绝对路径 train.txt> --max_sentence_len 100 
#    --embedding_size 100 --learning_rate 0.05  代码再执行的时候将会按照上面的参数来运行程序 
  
import tensorflow as tf 
  
FLAGS = tf.app.flags.FLAGS 
  
# tf.app.flags.DEFINE_string("param_name", "default_val", "description") 
tf.app.flags.DEFINE_string("train_data_path", "/desktop/train.txt", "training data dir") 
tf.app.flags.DEFINE_string("log_dir", "./logs", " the log dir") 
tf.app.flags.DEFINE_integer("max_sentence_len", 80, "max num of tokens per query") 
tf.app.flags.DEFINE_integer("embedding_size", 50, "embedding size") 
  
tf.app.flags.DEFINE_float("learning_rate", 0.001, "learning rate") 
  
def main(unused_argv): 
    train_data_path = FLAGS.train_data_path 
    print("train_data_path", train_data_path) 
    print("*" * 30) 
    max_sentence_len = FLAGS.max_sentence_len 
    print("max_sentence_len", max_sentence_len) 
    print("*" * 30) 
    embdeeing_size = FLAGS.embedding_size 
    print("embedding_size", embdeeing_size) 
    print("*" * 30) 
    abc = tf.add(max_sentence_len, embdeeing_size) 
  
    init = tf.global_variables_initializer() 
  
    with tf.Session() as sess: 
        sess.run(init) 
        print("abc", sess.run(abc)) 
if __name__ == '__main__': 
    tf.app.run()   # 解析命令行参数，调用main 函数 main(sys.argv) 
 

train_data_path /desktop/train.txt
******************************
max_sentence_len 80
******************************
embedding_size 50
******************************
abc 130


SystemExit: 

E:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [49]:
t = tf.nn.l2_loss([1., 2., 3.])
sess = tf.Session()
sess.run(t)
#( 1**2 + 2**2 + 3**2 )/ 2 = 14/2 = 7 

7.0

In [50]:
sess = tf.Session()
sess.run(t)

7.0

In [17]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys
def load_data(dataset_str):
    """
    Loads input data from gcn/data directory

    ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instances
        (a superset of ind.dataset_str.x) as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.y => the one-hot labels of the labeled training instances as numpy.ndarray object;
    ind.dataset_str.ty => the one-hot labels of the test instances as numpy.ndarray object;
    ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx as numpy.ndarray object;
    ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]} as collections.defaultdict
        object;
    ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting as list object.

    All objects above must be saved using python pickle module.

    :param dataset_str: Dataset name
    :return: All data input files loaded (as well the training/test data).
    """
    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []
    for i in range(len(names)):
        with open("data/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x, y, tx, ty, allx, ally, graph = tuple(objects)
    test_idx_reorder = parse_index_file("data/ind.{}.test.index".format(dataset_str))
    test_idx_range = np.sort(test_idx_reorder)

    if dataset_str == 'citeseer':
        # Fix citeseer dataset (there are some isolated nodes in the graph)
        # Find isolated nodes, add them as zero-vecs into the right position
        test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
        tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
        tx_extended[test_idx_range-min(test_idx_range), :] = tx
        tx = tx_extended
        ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
        ty_extended[test_idx_range-min(test_idx_range), :] = ty
        ty = ty_extended

    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    labels = np.vstack((ally, ty))
    labels[test_idx_reorder, :] = labels[test_idx_range, :]

    idx_test = test_idx_range.tolist()
    idx_train = range(len(y))
    idx_val = range(len(y), len(y)+500)

    train_mask = sample_mask(idx_train, labels.shape[0])
    val_mask = sample_mask(idx_val, labels.shape[0])
    test_mask = sample_mask(idx_test, labels.shape[0])

    y_train = np.zeros(labels.shape)
    y_val = np.zeros(labels.shape)
    y_test = np.zeros(labels.shape)
    y_train[train_mask, :] = labels[train_mask, :]
    y_val[val_mask, :] = labels[val_mask, :]
    y_test[test_mask, :] = labels[test_mask, :]

    return adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask
def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index
def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

In [18]:
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data("cora")

In [23]:
print(train_mask)

[ True  True  True ... False False False]


In [40]:
import tensorflow as tf
mask = tf.cast(train_mask, dtype=tf.float32)
p = tf.reduce_mean(mask)
mask /= tf.reduce_mean(p)
sess = tf.Session()
t = sess.run(mask)

In [41]:
print(t)

[19.342857 19.342857 19.342857 ...  0.        0.        0.      ]


In [5]:
new_adj = adj.toarray()
print(new_adj)
print(new_adj.shape)
# 如果一行由“论文1 论文2”表示，则链接是“论文2 - >论文1”。可以通过论文之间的索引关系建立邻接矩阵adj

[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
(2708, 2708)


In [6]:
new_features = features.toarray()
print(new_features)
print(new_features.shape)
# (2708, 1433) 2708篇论文  1433个关键词语 用关键词语作为每篇论文的特征
# with open("./features.txt", 'a') as file_handle :
#     for i in new_features :
#         for j in i :
#             file_handle.write(str(j))
#             file_handle.write(' ')
#         file_handle.write('\n')

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(2708, 1433)


In [7]:
new_y_train = y_train.tolist()

In [8]:
cou = 0
print(new_y_train[0])
for i in new_y_train :
    print(i)
    cou += 1
print(cou)

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [9]:
train_mask

array([ True,  True,  True, ..., False, False, False])

In [106]:
def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    # print(rowsum) # (2708, 1) [[ 9.][23.][19.]...[18.][13.]]
    r_inv = np.power(rowsum, -1).flatten()  # 1 / x
#     print(r_inv) # [0.11111111 0.04347826 0.05263158 ... 0.05555556 0.07142857 0.07692308] (2708,)
#     print(r_inv.shape)
    r_inv[np.isinf(r_inv)] = 0. # 如果刚刚发生除0错误，则置为 0 
    r_mat_inv = sp.diags(r_inv)  # 构造为对角线
#     print(r_mat_inv)
#     t = sp.diags([1, 2, 3])
#     t.toarray()
#     array([[1., 0., 0.],
#            [0., 2., 0.],
#            [0., 0., 3.]])
    features = r_mat_inv.dot(features) # 矩阵相乘
#     print(type(features))
#     写了这么多代码... 就是用广义度矩阵（一个点对应的一行的元素值(可能是特征值、出度/入度)，加起来作为这一个点的度）来进行归一化.
    return sparse_to_tuple(features)

def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        print(type(mx))
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
#         print(mx.row, mx.col ) # mx就是特征矩阵， mx.row 就是行坐标  mx.y 就是列坐标 
        coords = np.vstack((mx.row, mx.col)).transpose() # transpose矩阵的转置
        print(coords) # (49216, 2)  49216  相当于边的个数
        values = mx.data
        shape = mx.shape
        return coords, values, shape # coords 矩阵每个点的坐标  values 矩阵归一化之后每个点的大小 shape 矩阵的形状

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx

In [107]:
featurest = preprocess_features(features)

<class 'scipy.sparse.csr.csr_matrix'>
[0 0 0 0 0 0 0 0 0 1] [1274 1247 1194 ...  329  186   19]
[[   0 1274]
 [   0 1247]
 [   0 1194]
 ...
 [2707  329]
 [2707  186]
 [2707   19]]


In [17]:
featurest[2]

(2708, 1433)

In [45]:
t = sp.diags([1, 2, 3])
t.dot()
# for i in t.toarray:
#     for j in i :
#         print(j, end=' ')

TypeError: dot() missing 1 required positional argument: 'other'

In [14]:
class A(object) :
    def __init__(self, aa) :
        print(aa)
        self.name = "aa"
    def _print(self):
        print(self.name)
    def print(self):
        _print()

In [15]:
a = A

In [17]:
a._print

<function __main__.A._print>

In [37]:
import tensorflow as tf
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v = tf.get_variable("v2", [5])
assert v.name == "foo/bar/v2:0"

In [38]:
assert v.name == "foo/bar/v1:0"

AssertionError: 

In [21]:
assert 1 == 2

AssertionError: 

TypeError: norm() got an unexpected keyword argument 'shape'

In [14]:
class Test(object) :
    def __init__(self, name) :
        self.name = name
        print("name : " , name)
    def __call__(self, lala):
        print(self.name, " ", lala)


In [15]:
T = Test(123)

name :  123


In [16]:
T(456)

123   456


In [7]:
TT = T

In [8]:
TT(456)

name :  456
